In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import os, zipfile
import albumentations as A
from datetime import datetime
from PIL import Image


In [2]:

dir_name = './'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file


In [3]:
"""
path = "/content/drive/MyDrive/VisDrone2019-DET-test-dev"
image, box, label, p = augumentation(path, dict_labels)
"""

'\npath = "/content/drive/MyDrive/VisDrone2019-DET-test-dev"\nimage, box, label, p = augumentation(path, dict_labels)\n'

## Check and Delete

In [4]:
"""
Function for validate boxes trasformed to albumentations format.

check_bbox():
    * seq: array of boxes
    * W: weight image
    * H: Height image
"""
def check_bbox(seq, W, H):
    bbox_albu = A.core.bbox_utils.convert_bboxes_to_albumentations(seq, source_format='yolo', rows=H, cols=W)
    bbox_yolo = A.core.bbox_utils.convert_bboxes_from_albumentations(bbox_albu, target_format='yolo', rows=H, cols=W)
    return np.array(bbox_yolo)


"""
Function for detele all box near edge of image.

delete_noise():
    * transformed_bboxes: array of boxes
    * transformed_class_labels: array
"""
def delete_noise(transformed_bboxes, transformed_class_labels):
    idx = []
    for i in range(transformed_bboxes.shape[0]):
        x = transformed_bboxes[i][0]
        y = transformed_bboxes[i][1]
        w = transformed_bboxes[i][2]
        h = transformed_bboxes[i][3]
        th1 = x-w/2
        th2 = y-h/2
        
        if th1 < 0.03 or th1 > 0.95 or th2 < 0.03 or th2 > 0.95:
            idx.append(i)

    new_box = np.delete(transformed_bboxes, idx, axis=0)
    new_lab = np.delete(transformed_class_labels, idx, axis=0)

    return new_box, new_lab

    

In [5]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomCrop(p=1,width=500, height=500),
    A.RandomBrightnessContrast(p=0.1),
    A.RandomSnow(p=0.1),
    A.RandomRain(rain_type='heavy'),
    A.RandomRotate90(),
    A.RandomFog(fog_coef_lower=0.1, fog_coef_upper=0.2),
    
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

dict_labels = {
      0: "pedestrian",
      1: "people",
      2: "bicycle",
      3: "car",
      4: "van",
      5: "truck",
      6: "tricycle",
      7: "awning-tricycle",
      8: "bus",
      9: "motor"
  }

## Def Augumentation

In [12]:
#path: test or train which should contain 2 folder: /labels and /images

def augumentation(path, dict_labels, class_aug):
  
  for img in os.listdir(path + "/labels"):

    img_abs = img.split(".txt")[0]
    image = cv2.imread(path + "/images/" + img_abs +".jpg", cv2.COLOR_BGR2RGB) #open image
    with open(path + "/labels/" + img, "r") as file:
      line = [[num for num in line.replace("\n"," ").split(" ")] for line in file] #save labels
    
    #preprocessing data  
    line = np.array(line)
    line = line[:, :-1]
    opt = list(line[:,0])
    tot = filter(lambda x : x == str(class_aug), opt)
    #print(f"per {img} ho {len(list(tot))} elementi di {class_aug}")

    if str(class_aug) in opt:
      for i in range(len(list(tot))):
        print(i)
        np.set_printoptions(suppress=True)
        boxes = np.asarray(line, dtype=float)
        labels = [int(label) for label in boxes[:,0]]
        labels = np.array(labels)
        class_labels = [dict_labels[l] for l in labels]
        boxes =  boxes[:,1:]
        boxes_ = check_bbox(boxes, 500, 500) #check
        
        flag, new_image, new_boxes, new_labels, new_labels_id = balance_datatset(image=image, 
                                                                                  boxes=boxes_, 
                                                                                  labels=class_labels, 
                                                                                  class_aug=class_aug, 
                                                                                  flag=True)
        
        if flag:
          print(f"sono arrivato a 100 trasformazioni e non trovo nulla")
          i = i + 1
          continue
        else:
          if len(list(new_labels_id[new_labels_id[:]==class_aug])) != 0:
            print("entro qui", len(new_labels_id))
            new_labels_id_ = np.reshape(new_labels_id, (len(new_labels_id),1))
            temp = np.concatenate((new_labels_id_, new_boxes), axis = 1)
            output = temp[temp[:,0] == class_aug]
            dt = datetime.now()
            file_labels = f"{path}/labels/{dt.microsecond}.txt"
            file_image = f"{path}/images/{dt.microsecond}.jpg"
            np.savetxt(file_labels ,output, fmt=['%d', '%f', '%f', '%f', '%f'])
            im = Image.fromarray(new_image)
            im.save(file_image)
            i = i + 1
          else:
            i = i + 1
            print("not match")
          i = i + 1
      
    else:
      print(f"not first match in {img}")
      continue
  
    
  
  
      
    
  

In [7]:
def transformation(image, boxes, labels):

    transformed = transform(image=image, bboxes=boxes, class_labels=labels) 
    transformed_image = np.array(transformed['image'])
    transformed_bboxes = np.array(transformed['bboxes'])
    transformed_class_labels = np.array(transformed['class_labels'])
    transformed_bboxes_, transformed_class_labels_ =  delete_noise(transformed_bboxes, transformed_class_labels)
    transformed_labels_id = np.array([list(dict_labels.values()).index(i) for i in transformed_class_labels_ ])


    return transformed_image, transformed_bboxes_, transformed_class_labels, transformed_labels_id


In [8]:
def balance_datatset(image, boxes, labels, class_aug, flag=True):
    count = 0
    end = 100
    new_image, new_boxes, new_labels, new_labels_id = transformation(image, boxes, labels)
    condition = class_aug in new_labels_id
    while (count < end) and (not condition) and (new_labels_id.shape != 0):
        count = count + 1
        new_image, new_boxes, new_labels, new_labels_id = transformation(image, boxes, labels)
        flag = True
    if count == end or new_labels_id.shape == 0:
        flag = False
        
    return flag, new_image, new_boxes, new_labels, new_labels_id

In [13]:
augumentation("./VisDrone2019-DET-test-dev", dict_labels=dict_labels, class_aug=5)
#visualize(crop_image, new_b, new_l, dict_labels)

0
not match
1
not match
2
not match
3
not match
4
entro qui 2
5
entro qui 2
6
sono arrivato a 100 trasformazioni e non trovo nulla
7
sono arrivato a 100 trasformazioni e non trovo nulla
8
entro qui 2
9
not match
10
not match
11
sono arrivato a 100 trasformazioni e non trovo nulla
12
sono arrivato a 100 trasformazioni e non trovo nulla
13
sono arrivato a 100 trasformazioni e non trovo nulla
14
entro qui 2
0
entro qui 12
1
not match
2
entro qui 13
3
not match
4
entro qui 16
5
entro qui 12
6
sono arrivato a 100 trasformazioni e non trovo nulla
7
not match
8
entro qui 11
9
entro qui 12
10
not match
0
sono arrivato a 100 trasformazioni e non trovo nulla
1
sono arrivato a 100 trasformazioni e non trovo nulla
2
sono arrivato a 100 trasformazioni e non trovo nulla
3
sono arrivato a 100 trasformazioni e non trovo nulla
4
sono arrivato a 100 trasformazioni e non trovo nulla
5
sono arrivato a 100 trasformazioni e non trovo nulla
6
sono arrivato a 100 trasformazioni e non trovo nulla
7
sono arriva

In [10]:
a = np.array([5,5,5,6])
len(list(a[a[:] == 5]))

3

## Draw Image

In [11]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    dh, dw, _ = img.shape
    x,y,w,h = bbox
    l = int((x - w / 2) * dw)
    r = int((x + w / 2) * dw)
    t = int((y - h / 2) * dh)
    b = int((y + h / 2) * dh)
    
    if l < 0:
        l = 0
    if r > dw - 1:
        r = dw - 1
    if t < 0:
        t = 0
    if b > dh - 1:
        b = dh - 1

    cv2.rectangle(img, (l, t), (r, b), color=color, thickness=thickness)
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.putText(
          img,
          text=class_name,
          org=(l, t - int(0.3 * text_height)),
          fontFace=cv2.FONT_HERSHEY_SIMPLEX,
          fontScale=0.65, 
          color=TEXT_COLOR, 
          lineType=cv2.LINE_AA,
      )
   
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)